In [1]:
import tensorflow as tf
sess = tf.Session()



In [2]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True

In [3]:
import os

import numpy as np

from imp import reload



from keras.layers.merge import concatenate
from keras.models import Model


import sys


Using TensorFlow backend.


In [4]:


train_data_file =  "C:/input/train.csv/train.csv"
test_data_file = "C:/input/test.csv/test.csv"


In [5]:
#index word vectors
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format("C:/input/GoogleNews-vectors-negative300.bin", binary = True)


C:\Users\Shrey u\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
#text cleaning

def clean_text(text):
    
    #convert words in text to lower case and split them into words
    text = text.lower().split()
    
    #remove stopwords from the text
    stop = set(stopwords.words("english"))
    text = [w for w in text if not w in stop]
    
    text = " ".join(text)
    
    # clean text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
   
    

    return text


In [ ]:
import codecs
import csv
import re
from nltk.corpus import stopwords
from string import punctuation

train_texts_1 = [] 
train_texts_2 = []
labels = []

with codecs.open(train_data_file, encoding = 'utf-8') as f:
    reader = csv.reader(f, delimiter = ',')
    header = next(reader)
    for row in reader:
        train_texts_1.append(clean_text(row[3]))
        train_texts_2.append(clean_text(row[4]))
        labels.append(int(row[5]))
      
#train_texts_1 is a list containing question1 of training data
# similarily train_texts_2 contains question 2
#labels is a list containing 'is_duplicate'column from training data
                        

    

In [ ]:
test_texts_1 = []
test_texts_2 = []
test_ids = []

with codecs.open(test_data_file, encoding = 'utf-8') as f:
    reader = csv.reader(f, delimiter = ',')
    header = next(reader)
    for row in reader:
        test_texts_1.append(clean_text(row[1]))
        test_texts_2.append(clean_text(row[2]))
        test_ids.append(row[0])
 #likewise above cell test_texts_1, tests_texts_2 and test_ids are the list containing question1, question                   

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_words = 200000
max_seq_len = 30
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(train_texts_1 + train_texts_2 + test_texts_1 + test_texts_2)

train_sequences_1 = tokenizer.texts_to_sequences(train_texts_1)
train_sequences_2 = tokenizer.texts_to_sequences(train_texts_2)

test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)

word_index = tokenizer.word_index

train_data_1 = pad_sequences(train_sequences_1, maxlen = max_seq_len)
train_data_2 = pad_sequences(train_sequences_2, maxlen = max_seq_len)
labels = np.array(labels)

test_data_1 = pad_sequences(test_sequences_1, maxlen = max_seq_len)
test_data_2 = pad_sequences(test_sequences_2, maxlen = max_seq_len)
test_ids = np.array(test_ids)

In [ ]:
# prepare word embedding

e_words = min(max_words, len(word_index)) + 1
embedding_dim = 300

embedding_matrix = np.zeros((e_words, embedding_dim))
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)

In [ ]:
validation_split = 0.1

# split to train and validation set

perm = np.random.permutation(len(train_data_1))
idx_train = perm[:int(len(train_data_1) * (1 - validation_split))]
idx_val = perm[int(len(train_data_1) * (1 - validation_split)):]

X1_train = train_data_1[idx_train]
X2_train = train_data_2[idx_train]
y_train = labels[idx_train]

X1_val = train_data_1[idx_val]
X2_val = train_data_2[idx_val]
y_val = labels[idx_val]

X1_test = test_data_1
X2_test = test_data_2

# split to dicts

X_train = {'left' : X1_train, 'right' : X2_train}
X_val = {'left' : X1_val, 'right' : X2_val}
X_test = {'left' : X1_test, 'right': X2_test}

# make sure everything is good

assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(y_train)




In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Lambda, CuDNNLSTM
from keras.layers.merge import concatenate
import keras.backend as K

# initialize model variables

num_lstm = np.random.randint(175, 275)
batch_size = 2048
n_epoch = 30
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25

# function to check similarity between 2 outputs of lstm layer

def sim_func(left, right):
    ''' this function will estimate similarity between the outputs of lstm'''
    return K.exp(-K.sum(K.abs(left - right), axis = 1, keepdims = True))
# define the model structure

# the visible layer

left_input = Input(shape = (max_seq_len,), dtype = 'int32')
right_input = Input(shape = (max_seq_len,), dtype = 'int32')

# the embedding layer

embedding_layer = Embedding(e_words, embedding_dim, 
                            weights = [embedding_matrix],
                            input_length = max_seq_len,
                            trainable = False )
# embedded version of the input

encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# since, this is a siamese network both the sub network share same lstm 

shared_lstm = LSTM(num_lstm)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)




malstm_distance = Lambda(lambda x: sim_func(x[0], x[1]),
                         output_shape = lambda x: (x[0][0], 1))([left_output, right_output])
preds = Dense(1, activation='sigmoid')(malstm_distance)

# pack it all into a model

malstm = Model([left_input, right_input], [preds])

malstm.compile(loss = 'binary_crossentropy',
              optimizer = 'nadam',
              metrics = ['acc'])


In [ ]:
# start training

from time import time

training_start_time = time()

malstm_trained = malstm.fit([X_train['left'], X_train['right']], y_train, batch_size = batch_size, epochs = n_epoch,
                           validation_data = ([X_val['left'], X_val['right']], y_val))

#print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

In [ ]:
# Plot accuracy
import matplotlib.pyplot as plt

plt.plot(malstm_trained.history['acc'])
plt.plot(malstm_trained.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(malstm_trained.history['loss'])
plt.plot(malstm_trained.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# prediction using test set
preds = malstm.predict([X_test['left'], X_test['right']], batch_size = 3000, verbose = 1)


In [ ]:
submission = pd.DataFrame({'test_id':test_ids, 'is_duplicate':preds.ravel()})

submission.to_csv('pred.csv', index=False)
